In [3]:

from datetime import datetime, timedelta
from textwrap import dedent

# The DAG object; we'll need this to instantiate a DAG
from airflow import DAG

# Operators; we need this to operate!
from airflow.operators.bash import BashOperator
with DAG(
    'tutorial',
    # These args will get passed on to each operator
    # You can override them on a per-task basis during operator initialization
    default_args={
        'depends_on_past': False,
        'email': ['airflow@example.com'],
        'email_on_failure': False,
        'email_on_retry': False,
        'retries': 1,
        'retry_delay': timedelta(minutes=5),
        # 'queue': 'bash_queue',
        # 'pool': 'backfill',
        # 'priority_weight': 10,
        # 'end_date': datetime(2016, 1, 1),
        # 'wait_for_downstream': False,
        # 'sla': timedelta(hours=2),
        # 'execution_timeout': timedelta(seconds=300),
        # 'on_failure_callback': some_function,
        # 'on_success_callback': some_other_function,
        # 'on_retry_callback': another_function,
        # 'sla_miss_callback': yet_another_function,
        # 'trigger_rule': 'all_success'
    },
    description='A simple tutorial DAG',
    schedule_interval=timedelta(days=1),
    start_date=datetime(2021, 1, 1),
    catchup=False,
    tags=['example'],
) as dag:

    # t1, t2 and t3 are examples of tasks created by instantiating operators
    t1 = BashOperator(
        task_id='print_date',
        bash_command='date',
    )

    t2 = BashOperator(
        task_id='sleep',
        depends_on_past=False,
        bash_command='sleep 5',
        retries=3,
    )
    t1.doc_md = dedent(
        """\
    #### Task Documentation
    You can document your task using the attributes `doc_md` (markdown),
    `doc` (plain text), `doc_rst`, `doc_json`, `doc_yaml` which gets
    rendered in the UI's Task Instance Details page.
    ![img](http://montcs.bloomu.edu/~bobmon/Semesters/2012-01/491/import%20soul.png)

    """
    )

    dag.doc_md = __doc__  # providing that you have a docstring at the beginning of the DAG
    dag.doc_md = """
    This is a documentation placed anywhere
    """  # otherwise, type it like this
    templated_command = dedent(
        """
    {% for i in range(5) %}
        echo "{{ ds }}"
        echo "{{ macros.ds_add(ds, 7)}}"
    {% endfor %}
    """
    )

    t3 = BashOperator(
        task_id='templated',
        depends_on_past=False,
        bash_command=templated_command,
    )

    t1 >> [t2, t3]

In [7]:
# initialize the database tables
!airflow db init

# print the list of active DAGs
!airflow dags list

# prints the list of tasks in the "tutorial" DAG
!airflow tasks list tutorial

# prints the hierarchy of tasks in the "tutorial" DAG
!airflow tasks list tutorial --tree


DB: sqlite:///C:\Users\alima/airflow/airflow.db
[2022-05-15 11:40:03,652] {db.py:1410} INFO - Creating tables


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Traceback (most recent call last):
  File "C:\Users\alima\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\alima\anaconda3\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\alima\anaconda3\Scripts\airflow.exe\__main__.py", line 7, in <module>
  File "C:\Users\alima\anaconda3\lib\site-packages\airflow\__main__.py", line 38, in main
    args.func(args)
  File "C:\Users\alima\anaconda3\lib\site-packages\airflow\cli\cli_parser.py", line 51, in command
    return func(*args, **kwargs)
  File "C:\Users\alima\anaconda3\lib\site-packages\airflow\cli\commands\db_command.py", line 35, in initdb
    db.initdb()
  File "C:\Users\alima\anaconda3\lib\site-packages\airflow\utils\session.py", line 71, in wrapper
    return func(*args, session=session, **kwargs)
  File "C:\

[2022-05-15 11:40:11,885] {cli_action_loggers.py:105} WARNING - Failed to log action with (sqlite3.OperationalError) no such table: log
[SQL: INSERT INTO log (dttm, dag_id, task_id, event, execution_date, owner, extra) VALUES (?, ?, ?, ?, ?, ?, ?)]
[parameters: ('2022-05-15 09:40:11.881313', 'tutorial', None, 'cli_task_list', None, 'alima', '{"host_name": "LAPTOP-M051T6BL", "full_command": "[\'C:\\\\\\\\Users\\\\\\\\alima\\\\\\\\anaconda3\\\\\\\\Scripts\\\\\\\\airflow\', \'tasks\', \'list\', \'tutorial\']"}')]
(Background on this error at: http://sqlalche.me/e/14/e3q8)
print_date
sleep
templated


[2022-05-15 11:40:14,090] {cli_action_loggers.py:105} WARNING - Failed to log action with (sqlite3.OperationalError) no such table: log
[SQL: INSERT INTO log (dttm, dag_id, task_id, event, execution_date, owner, extra) VALUES (?, ?, ?, ?, ?, ?, ?)]
[parameters: ('2022-05-15 09:40:14.086503', 'tutorial', None, 'cli_task_list', None, 'alima', '{"host_name": "LAPTOP-M051T6BL", "full_command": "[\'C:\\\\\\\\Users\\\\\\\\alima\\\\\\\\anaconda3\\\\\\\\Scripts\\\\\\\\airflow\', \'tasks\', \'list\', \'tutorial\', \'--tree\']"}')]
(Background on this error at: http://sqlalche.me/e/14/e3q8)
<Task(BashOperator): print_date>
    <Task(BashOperator): sleep>
    <Task(BashOperator): templated>
